In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys

sys.path.append("../src")

from detector import MotionDetector

In [3]:
import os
from pathlib import Path

current_path = Path(os.getcwd())
project_path = current_path.parent
new_path = project_path / 'resources'

PATH_DATA = new_path

PATH_VIDEO = f"{PATH_DATA}/motion_video.avi"

In [4]:
detector = MotionDetector(
    lower_bound=np.array([64, 84, 75]),
    upper_bound=np.array([92, 255, 255]),
    roi_algo="contours",
    contours_coords=[[639, 436], [464, 379], [200, 380], [0, 447], [0, 479], [170, 400], [482, 395], [636, 465]],
)

In [5]:
cap = cv2.VideoCapture(PATH_VIDEO)

num_frame = 0

previous_frame = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    num_frame += 1
    
    res = detector.detect(frame)
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)
    
    if previous_frame is None:
        previous_frame = gray_frame
        continue
    
    frame_diff = cv2.absdiff(previous_frame, gray_frame)
    
    _, motion_mask = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
    
    previous_frame = gray_frame
    
    if detector.is_roi_found():
        features = []
        zones = detector.zones
        
        # roi_display = np.zeros_like(gray_frame)
        roi_display = frame.copy()
        
        for zone in zones:
            if "roi_mask" not in zone or zone["roi_mask"] is None:
                features.appen(0)
                continue
            
            rect = zone["roi_rect"]
            mask = zone["roi_mask"]
            
            x, y, w, h = rect
            
            roi = motion_mask[y : y + h, x : x + w]
            motion_value = np.sum(roi) / 255  # Количество пикселей с движением
            features.append(motion_value)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            roi_gray = cv2.cvtColor(roi, cv2.COLOR_GRAY2BGR)
            
            roi_display[y:y+h, x:x+w] = roi_gray
            
        frame = np.hstack((frame, roi_display))
    
    cv2.imshow("Frame with zones", frame)
    
    if cv2.waitKey(4) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
                